In [2]:
import pandas as pd
import glob


In [3]:
raw_path = 'Raw_csv\\'
normalized_path = 'Normalized_csv\\'
pre_normalized_path = 'Pre_Normalized_csv\\'
powered_columns = ['Name', 'Level', 'Price', 'EAC Bonus', 'KAC Bonus','Maximum Dex Bonus',
                   'Armor Check Penalty', 'Speed Adjustment', 'Upgrade Slots', 'Bulk', 
                   'Proficiency'] + ['Source', 'Description', 'Speed', 'Strength', 'Damage', 'Size', 
                    'Capacity', 'Usage', 'Weapon Slots']
melee_weapons_columns = ['Name', 'Category', 'Level', 'Price', 'Damage', 'Range', 'Critical',
       'Capacity', 'Usage', 'Bulk', 'Special']
medicinal_columns = ['Name', 'Category', 'Level', 'Price', 'Bulk']

In [12]:

def norm_armors(df, proficiency):
    if proficiency == 'Powered':
        df['Speed Adjustment'] = ['—']*len(df)
        df.rename(columns={'Item Level':'Level','Max Dex Bonus':'Maximum Dex Bonus' }, inplace=True)
    df['Armor Check Penalty'] = df['Armor Check Penalty'].replace('—', '0')
    df['Speed Adjustment'] = df['Speed Adjustment'].map({'—': 0, '-5 ft.':-1, '-10 ft.':-2})
    df['Proficiency'] = [proficiency]*len(df)
    return df

def norm_melee_weapons(df):
    df['Range'] = df.apply(lambda row: row['Special'].lower().split('thrown (')[1][:6] 
                           if 'thrown (' in row['Special'].lower() 
                           else '-', axis=1)
    df['Special'] = ['; usage'.join(text.split(', usage')) for text in df['Special']]
    df['Capacity'] = df.apply(lambda row: row['Special'].lower().split('; usage ')[0][-2:].strip('(') 
                              if '; usage ' in row['Special'].lower() 
                              else '-', axis=1)
    df['Usage'] = df.apply(lambda row: row['Special'].lower().split('; usage ')[1][0] 
                           if '; usage ' in row['Special'].lower() 
                           else '-', axis=1)
    df['Capacity'] = df.apply(lambda row: row['Capacity']+' charges' 
                              if 'powered' in row['Special'].lower() 
                              else row['Capacity'], axis=1)
    df['Capacity'] = df.apply(lambda row: row['Capacity']+' shell' 
                              if row['Capacity'][0]=='1' 
                              else row['Capacity'], axis=1)
    df['Capacity'] = df.apply(lambda row: row['Capacity']+' shells' 
                              if 'shell' not in row['Capacity'] and 'shells' in row['Special'] 
                              else row['Capacity'], axis=1)
    df['Special'] = [','.join([part.lower() for part in text.split(',') if 'usage' not in part]) 
                     for text in df['Special']]
    df['Special'] = df['Special'].replace('thrown \(.. ft.\)', 'thrown', regex=True)
    df = df[melee_weapons_columns]
    return df

def norm_grenades(df):
    df['Usage'] = ['-']*len(df)
    df['Damage'] = ['-']*len(df)
    df['Category'] = ['-']*len(df)
    df['Critical'] = ['-']*len(df)
    df = df[melee_weapons_columns]
    return df

def norm_weapons(df, proficiency, hands):
    df['Proficiency'] = [proficiency]*len(df)
    df['Hands'] = [hands]*len(df)
    return df

def clean_df_old(df):
    df['Bulk'] = df['Bulk'].replace('—', 0)
    df['Bulk'] = df['Bulk'].replace('L', 0.1)
    df['Price'] = df['Price'].replace('—', 0)
    return df

def save_df_to_csv_old(df, pathfile):
    df.to_csv(pathfile, sep='|',index=False)
    
def normalize_items(item_file):
    df = pd.read_csv(item_file, delimiter="	")
    filename = item_file.split('_Raw.')[0].split('\\')[-1]
    if 'Weapons' in filename:
        if 'Melee' in filename:
            df = norm_melee_weapons(df)
        if 'Grenades' in filename:
            df = norm_grenades(df)
        if '1Hd' in filename or 'Small_Arms' in filename:
            hands = 1
        else:
            hands = 2
        proficiency = ' '.join(filename.split('_')[:3]).strip()
        df = norm_weapons(df, proficiency, hands)
    elif 'Armors' in filename:
        proficiency = ' '.join(filename.split('_')[:1])
        df = norm_armors(df, proficiency)
    elif 'Armor_Upgrades' in filename:
        df['Prerequisite'] = df['Armor Type'].replace('Any', 'Light, Heavy, Powered')
    df = clean_df(df)
    if 'Powered' in filename:
        df = df[powered_columns]
    save_df_to_csv(df, normalized_path+filename+'.csv')
    print(filename + ' normalized')

def normalize_items2(item_file):
    df = pd.read_csv(item_file, delimiter="	")
    filename = item_file.split('_Raw2.')[0].split('\\')[-1]
    if 'Medicinal' in filename:
        df['Bulk'] = [0]*len(df)
        df['Category'] = ['Medicinal']*len(df)
        df = df[medicinal_columns]
    df = clean_df(df)
    save_df_to_csv(df, pre_normalized_path+filename+'.csv')
    print(filename + ' normalized')


In [426]:
item_files = glob.glob(raw_path+'*Items_Raw.csv')
for item_file in item_files :
    normalize_items(item_file)
    

Advanced_Melee_Weapons_1Hd_Items normalized
Advanced_Melee_Weapons_2Hd_Items normalized
Ammunitions_Items normalized
Armor_Upgrades_Items normalized
Basic_Melee_Weapons_1HD_Items normalized
Basic_Melee_Weapons_2HD_Items normalized
Grenades___1Hd_Weapons_Items normalized
Heavy_Armors_Items normalized
Heavy_Weapons__Items normalized
Light_Armors_Items normalized
Long_Arms__Weapons_Items normalized
Powered_Armors_Items normalized
Small_Arms__Weapons_Items normalized
Sniper_Weapons__Items normalized
Solarian_Crystals_Items normalized
Special_Weapons__1Hd_Items normalized
Special_Weapons__2Hd_Items normalized
Weapon_Accessoires_Items normalized


In [15]:
item_files2 = glob.glob(raw_path+'*Items_Raw2.csv')
print(item_files2)
for item_file in item_files2 :
    normalize_items2(item_file)

['Raw_csv\\Food&Drinks&TradeGoods_Items_Raw2.csv', 'Raw_csv\\Hybrid_Items_Raw2.csv', 'Raw_csv\\Magic_Items_Raw2.csv', 'Raw_csv\\Medicinal_Items_Raw2.csv', 'Raw_csv\\Personal_Items_Raw2.csv', 'Raw_csv\\Technological_Items_Raw2.csv']
Food&Drinks&TradeGoods_Items normalized
Hybrid_Items normalized
Magic_Items normalized
Medicinal_Items normalized
Personal_Items normalized
Technological_Items normalized


In [4]:
item_files = glob.glob(raw_path+'*_Raw.csv')

print(item_files)


['Raw_csv\\Advanced_Melee_Weapons_Raw.csv', 'Raw_csv\\Ammunitions2_Raw.csv', 'Raw_csv\\Ammunitions_Raw.csv', 'Raw_csv\\Armor_Upgrades_Raw.csv', 'Raw_csv\\Augmentations_Biotech_Raw.csv', 'Raw_csv\\Augmentations_Cybernetics_Raw.csv', 'Raw_csv\\Augmentations_Magitech_Raw.csv', 'Raw_csv\\Augmentations_Necrografts_Raw.csv', 'Raw_csv\\Basic_Melee_Weapons_Raw.csv', 'Raw_csv\\Feats_100_Raw.csv', 'Raw_csv\\Feats_150_Raw.csv', 'Raw_csv\\Feats_End_Raw.csv', 'Raw_csv\\Feats_fifty_Raw.csv', 'Raw_csv\\Grenade_Weapons_Raw.csv', 'Raw_csv\\Heavy_Armors_Raw.csv', 'Raw_csv\\Heavy_Weapons_Raw.csv', 'Raw_csv\\Hybrid_Items_Raw.csv', 'Raw_csv\\Light_Armors_Raw.csv', 'Raw_csv\\Longarms_Weapons_Raw.csv', 'Raw_csv\\Magic_Items_Raw.csv', 'Raw_csv\\Medicinal_Items_Raw.csv', 'Raw_csv\\Personal_Items_Raw.csv', 'Raw_csv\\Powered_Armors_Raw.csv', 'Raw_csv\\Smallarms_Weapons_Raw.csv', 'Raw_csv\\Sniper_Weapons_Raw.csv', 'Raw_csv\\Solarian_Weapons_Raw.csv', 'Raw_csv\\Technological_Items_Raw.csv', 'Raw_csv\\Weapon_Access

In [77]:
def clean_df(df):
    df['Bulk'] = [item.strip(';')for item in df['Bulk']]
    df['Level'] = [item.strip(';')for item in df['Level']]
    df['Hands'] = [item.strip(';')for item in df['Hands']]
    df['Price'] = [item.strip(';')for item in df['Price']]
    df['Damage'] = [item.strip(';')for item in df['Damage']]
    df['Bulk'] = df['Bulk'].replace('—', 0)
    df['Bulk'] = df['Bulk'].replace('L', 0.1)
    df['Price'] = df['Price'].replace('—', 0)
    df['Description'] = df['Description'].replace('Description', '', regex=True)
    df['Description'] = df['Description']
    return df

def save_df_to_csv(df, pathfile):
    df.to_csv(pathfile, sep='|',index=False)

In [83]:
for file in item_files:
    if 'Feats' not in file:
        df = pd.read_csv(file, delimiter="|")
        clean_df(df)
        name = file.split('\\')[-1].split('_Raw')[0]
        print(name)
        save_df_to_csv(df, pre_normalized_path + name + '_Clean.csv')



Advanced_Melee_Weapons
Ammunitions2
Ammunitions


KeyError: 'Level'

In [79]:
clean_df(df)
df

,Name,Source,Level,Price,Hands,Proficiency,Damage,Critical,Capacity,Bulk,Special,Description,Nethys
0,Arrows,Starfinder Core Rulebook pg. 179,1,50,1,Ammo,—,—,20,0.1,—,Arrow shafts are made of carbon fiberâ€“reinfo...,https://www.aonsrd.com/WeaponDisplay.aspx?Item...
1,Darts,Starfinder Core Rulebook pg. 179,1,20,1,Ammo,—,—,25,0.1,—,These light metal shafts each have a pointed t...,https://www.aonsrd.com/WeaponDisplay.aspx?Item...
2,Flare,Starfinder Core Rulebook pg. 179,1,5,1,Ammo,—,—,1,0,—,"Usually made of magnesium, flares burn brightl...",https://www.aonsrd.com/WeaponDisplay.aspx?Item...
3,Flechette,Pact Worlds pg. 194,1,75,1,Ammo,—,—,25,0.1,—,NaN,https://www.aonsrd.com/WeaponDisplay.aspx?Item...
4,Scattergun Shells,Starfinder Core Rulebook pg. 179,1,55,1,Ammo,—,—,25,0.1,—,These cartridges are packed with small metal s...,https://www.aonsrd.com/WeaponDisplay.aspx?Item...
5,"Battery, Standard",Starfinder Core Rulebook pg. 179,1,60,1,Ammo,—,—,20,0,—,"Battery|Batteries charge powered weapons, but ...",https://www.aonsrd.com/WeaponDisplay.aspx?Item...
6,"Petrol Tank, Standard",Starfinder Core Rulebook pg. 179,1,60,1,Ammo,—,—,20,1,—,Petrol Tank|Petrol is a highly flammable blend...,https://www.aonsrd.com/WeaponDisplay.aspx?Item...
7,"Rounds, Longarm and Sniper",Starfinder Core Rulebook pg. 179,1,75,1,Ammo,—,—,25,0.1,—,"Rounds|Cased rounds are housed in magazines, w...",https://www.aonsrd.com/WeaponDisplay.aspx?Item...
8,"Rounds, Small Arm",Starfinder Core Rulebook pg. 179,1,40,1,Ammo,—,—,30,0.1,—,"Rounds|Cased rounds are housed in magazines, w...",https://www.aonsrd.com/WeaponDisplay.aspx?Item...
9,"Rounds, Heavy",Starfinder Core Rulebook pg. 179,2,90,1,Ammo,—,—,20,0.1,—,"Rounds|Cased rounds are housed in magazines, w...",https://www.aonsrd.com/WeaponDisplay.aspx?Item...


In [61]:
name = filename.split('\\')[-1].split('_Raw')[0]
save_df_to_csv(df, pre_normalized_path + name + '_Clean.csv')
    